## Script to create Gowalla dataset containing 80 Users with 200 movements each 

In [5]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.python.client import device_lib
from keras.backend import set_session
import os
import random

In [6]:
device_lib.list_local_devices()

2022-12-02 01:43:52.016637: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 01:43:52.016826: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 01:43:52.016925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 01:43:52.017011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 01:43:52.017096: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6061656794223327297
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 8480882688
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3363689186013438558
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 23430234112
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9251352152290873875
 physical_device_desc: "device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:02:00.0, compute capability: 8.6"
 xla_global_id: 2144165316]

In [7]:
df = pd.read_csv('../datasets_produced/gowalla/dataset.csv')
df.head()

,uid,datetime,lat,lng,vid
0,0,2010-10-19 23:55:27,30.235909,-97.795140,22847
1,0,2010-10-18 22:17:43,30.269103,-97.749395,420315
2,0,2010-10-17 23:42:03,30.255731,-97.763386,316637
3,0,2010-10-17 19:26:05,30.263418,-97.757597,16516
4,0,2010-10-16 18:50:42,30.274292,-97.740523,5535878


In [8]:
groups = df.groupby('uid')
sizes = groups.size()
min_checkins = 200
users = sizes[sizes>=min_checkins].index 
df_min_checkins = df[df['uid'].isin(users)]
n_rows = list(range(0,200))
df_min_checkins_max_users = df_min_checkins.groupby('uid').nth(n_rows)
df_final = df_min_checkins_max_users.reset_index()
users_final = random.choices(users, k=80)
result_df = df_final[df_final.uid.isin(users_final)]
result_df['datetime'] = pd.to_datetime(result_df.datetime).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
result_df = result_df.sort_values(['uid', 'datetime'], ascending = (True, False))
vocabulary = list(result_df.vid.unique())
vid_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_vid = dict((i, c) for i, c in enumerate(vocabulary))
newList = [vid_indices[k] for k in result_df.vid if k in vid_indices]
result_df.vid = newList
result_df = result_df.astype({'vid':'int'})
result_df.to_csv(r'../datasets_final/gowalla_random_80_200/dataset.txt', sep='\t', mode='a', header=False, index=False)
result_df
# df_final

/tmp/ipykernel_2386855/2670038292.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['datetime'] = pd.to_datetime(result_df.datetime).dt.strftime('%Y-%m-%dT%H:%M:%SZ')


,uid,datetime,lat,lng,vid
9200,140,2010-10-17T19:15:53Z,38.950992,-104.803644,0
9201,140,2010-10-09T03:24:01Z,40.019661,-105.273351,1
9202,140,2010-10-08T21:03:28Z,40.018910,-105.275352,2
9203,140,2010-10-08T19:20:25Z,40.019186,-105.277197,3
9204,140,2010-09-27T04:55:55Z,38.881179,-104.718561,4
...,...,...,...,...,...
1292795,153219,2010-08-29T17:10:14Z,62.147430,6.095411,9519
1292796,153219,2010-08-29T15:24:31Z,62.144819,6.098667,9544
1292797,153219,2010-08-27T20:44:51Z,62.147430,6.095411,9519
1292798,153219,2010-08-27T07:21:24Z,62.149562,6.076813,9507
